# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd  #To Import data
import numpy as np   #For Numerical analysis
import matplotlib.pyplot as plt #Data plotting and visualization
import seaborn as sns #For Data visualization


import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.multioutput import MultiOutputClassifier

In [4]:
# load data from database
from sqlalchemy import create_engine


engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM InsertTableName", engine)
df = df[df['related'] != 2]
X = df['message']
y = df.iloc[:, 4:]

    
category_names = list(df.columns[4:])
category_names 

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [ ]:
y

In [6]:
df['genre'].value_counts()
df['related'].value_counts()


1    19906
0     6122
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [7]:
from nltk.tokenize import word_tokenize

#Regex to detect URL
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

#Function to tokenize the text
def tokenize(text):
    detected_urls = re.findall(url_regex , text)
    
    for url in detect_urls:
        text = text.replace(url, "urlplaceholder")
        
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
        
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
vect = CountVectorizer(tokenizer=tokenize) #Transformers
tfidf = TfidfTransformer()#Transformer
clf = RandomForestClassifier()#Estimator




In [9]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('vect' , CountVectorizer()),
    ('tfidf' , TfidfTransformer()),
    ('clf' , MultiOutputClassifier(RandomForestClassifier()))
    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
from sklearn.model_selection import train_test_split

# load data
#X, y = load_data()

# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#Train classifier
pipeline.fit(X_train , y_train)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [27]:

#Predict on test data
y_pred = pipeline.predict(X_test)
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [29]:
X_test

21046    Light showers may bring additional relief duri...
12461    This place is pretty empty considering the sto...
25670    In this capacity, Ly covers the entire rural c...
13912    An aftershock measuring 5.5 occurred 90 minute...
14719    However, I must stress that the United Nations...
                               ...                        
23174    African countries had to strengthen their voic...
14989    Since 29 June, severe rainstorms have hit the ...
24556    Thankfully they escaped uninjured, but their h...
16801    The sea of Binh Dinh Province has been violent...
4699     I have an organization recognized by Social Af...
Name: message, Length: 8590, dtype: object

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
y_test['related'].value_counts()

1    6547
0    2043
Name: related, dtype: int64

In [25]:
#classification report

from sklearn.metrics import classification_report, accuracy_score


# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print('Category: {} '.format(category_names[i]))
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])))


Category: related 
              precision    recall  f1-score   support

           0       0.74      0.27      0.40      2043
           1       0.81      0.97      0.88      6547

    accuracy                           0.80      8590
   macro avg       0.78      0.62      0.64      8590
weighted avg       0.79      0.80      0.77      8590

Accuracy 0.8041909196740396


Category: request 
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      7126
           1       0.87      0.45      0.60      1464

    accuracy                           0.90      8590
   macro avg       0.89      0.72      0.77      8590
weighted avg       0.89      0.90      0.88      8590

Accuracy 0.8958090803259604


Category: offer 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8553
           1       0.00      0.00      0.00        37

    accuracy                           1.00      8590
   macro avg 


Accuracy 0.949592549476135


Category: electricity 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8410
           1       0.67      0.02      0.04       180

    accuracy                           0.98      8590
   macro avg       0.82      0.51      0.52      8590
weighted avg       0.97      0.98      0.97      8590

Accuracy 0.979278230500582


Category: tools 
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8541
           1       0.00      0.00      0.00        49

    accuracy                           0.99      8590
   macro avg       0.50      0.50      0.50      8590
weighted avg       0.99      0.99      0.99      8590

Accuracy 0.9942956926658906


Category: hospitals 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8499
           1       0.00      0.00      0.00        91

    accuracy                    

In [16]:
# Print classification report on test data
print(classification_report(y_test.iloc[:, 0:].values, np.array([x[0:] for x in y_pred]), target_names = category_names))
print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      6547
               request       0.87      0.45      0.60      1464
                 offer       0.00      0.00      0.00        37
           aid_related       0.80      0.62      0.70      3615
          medical_help       0.73      0.05      0.09       701
      medical_products       0.79      0.06      0.11       441
     search_and_rescue       0.88      0.03      0.05       264
              security       0.33      0.01      0.01       167
              military       0.65      0.04      0.07       289
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.29      0.44       564
                  food       0.85      0.50      0.63       968
               shelter       0.85      0.30      0.45       780
              clothing       0.80      0.06      0.12       129
                 money       0.82      

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [31]:
# Defining a sentiment analyser function
from textblob import TextBlob
def sentiment_analyser(text):
    return text.apply(lambda Text: pd.Series(TextBlob(Text).sentiment.polarity))

df_123 = sentiment_analyser(X_test)

ModuleNotFoundError: No module named 'textblob'

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.